<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

# Assignment 5: Deep Learning {-}

This assignment aims at familiarizing you with training and testing a Deep Neural Network (DNN). The dataset you will be working on is CIFAR-10. You will have to do:

1.  **(5 points) Coding tasks:** The following questions involve writing code to complete specific tasks.  
    1.1 *(1 point)* Load the CIFAR-10 dataset, visualize sample images, and perform data normalization to improve training performance.  
    1.2 *(1 point)* First network: Build, train, and test a deep neural network with at least three convolutional layers, two fully connected layers, and two pooling layers.  
    1.3 *(1 point)* Second network: Build, train, and test another deep neural network, with an architecture of your choice, but at most 4M (four million) parameters, ensuring the architecture meets this constraint by verifying with model.summary().  
    1.4 *(2 points)* Modify the second network architecture by tuning the layer hyperparameters or adjusting the layer design to improve test accuracy while remaining within the four million parameter limit. Discuss your observations and the trade offs of the changes you make.  

2.  **(5 points) Open discussion questions:** These discussion questions ask you to analyze and argue your points.  Feel free to include relevant code examples to strengthen your arguments.  
    2.1 *(1 point)* How did hyperparameter tuning (learning rate, dropout, batch size) affect your model’s accuracy? Were there any unexpected results?  
    2.2 *(1 point)* How did the constraint of keeping the model within 4 million parameters impact your design choices? Would a larger model necessarily perform better?  
    2.3 *(1 point)* How can deep learning models trained on datasets like CIFAR-10 be applied in real-world scenarios? Give an example.  
    2.4 *(1 point)* Deep learning models for image recognition can have biases. What ethical concerns should be considered when deploying such models?  
    2.5 *(1 point)* What was the most interesting or challenging part of this assignment? If you had more time, what additional improvements would you make?  

The dataset you will be working on is CIFAR-10 (https://www.cs.toronto.edu/~kriz/cifar.html) which consists of 60,000 32x32 colour images in 10 classes, with 6,000 images per class. There are 50,000 training images and 10,000 test images. Here follows the ten object classes:
* airplane
*	automobile
*	bird
*	cat
*	deer
*	dog
*	frog
*	horse
*	ship
*	truck

Here follows some data samples in the dataset:

![alt text](https://docs.pytorch.org/tutorials/_images/cifar10.png)

### Submission {-}
The structure of submission folder should be organized as follows:

- ./\<StudentID>-assignment5-notebook.ipynb: Jupyter notebook containing source code.
- ./\<Test-accuracy>-\<StudentID>.txt: accuracy of the second network on the test set (for extra credit, see the 'Evaluation' part below). For example if you get 0.8124 accuracy, the name of this file is 08124-2012345.txt. The file content is left empty.

The submission folder is named ML4DS-\<StudentID>-Assignment5 (e.g., ML4DS-2012345-Assigment5) and then compressed with the same name.
    
### Evaluation {-}
Assignment evaluation will be conducted on how you accomplish the assignment requirements. It is a plus if you have modeling steps other than the basic requirements and achieve an excellent model accuracy. In addition, your code should conform to a Python coding convention such as PEP-8.

EXTRA CREDIT: Top-3 submissions achieving the highest test accuracy on the second network (of 4M params at most) will be rewarded an extra credit. **You have to ensure the architecture meets this constraint by verifying and printing out the number of parameters with model.summary(). Please follow the submission format to be eligible for this extra credit.**

### Deadline {-}
Please visit Canvas for details.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras              # Keras is the high-level API of TensorFlow

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
# PLEASE DO NOT CHANGE THIS CODE

# Load the cifar10 dataset and split train/test
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Split train/valid from the training set
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=5)

print("Train shape: X_train = " + str(X_train.shape) + ", y_train = " + str(y_train.shape))
print("Validation shape: X_val = " + str(X_val.shape) + ", y_val = " + str(y_val.shape))
print("Test shape: X_test = " + str(X_test.shape) + ", y_test = " + str(y_test.shape))

In [ ]:
# Show some samples in the dataset
import matplotlib.pyplot as plt
imgplot = plt.imshow(X_train[44999])
plt.show()
imgplot = plt.imshow(X_test[4999])
plt.show()

## 1. Coding tasks

1. Load the CIFAR-10 dataset, visualize sample images, and perform data normalization to improve training performance.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# CIFAR-10 class names
class_names = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

# Count how many samples belong to each class
class_counts = np.bincount(y_train.flatten(), minlength=10)

# For each class, collect indices of all images
class_indices = {i: np.where(y_train.flatten() == i)[0] for i in range(10)}

# Plot setup
fig, axes = plt.subplots(4, 3, figsize=(6, 8))
axes = axes.flatten()

for i in range(10):
    ax = axes[i]

    # Randomly pick 4 images from this class
    idxs = np.random.choice(class_indices[i], 4, replace=False)
    imgs = X_train[idxs]

    # Create a small 2x2 grid inside the subplot
    combined = np.zeros((64, 64, 3), dtype=np.uint8)  # 32*2 = 64 pixels per side
    combined[:32, :32] = imgs[0]
    combined[:32, 32:] = imgs[1]
    combined[32:, :32] = imgs[2]
    combined[32:, 32:] = imgs[3]

    ax.imshow(combined)
    ax.set_title(f"{class_names[i]} ({class_counts[i]})", fontsize=10)
    ax.axis("off")

# Turn off remaining unused subplots
for ax in axes[10:]:
    ax.axis("off")

plt.show()


In [ ]:
# Convert pixel values from 0–255 to 0–1
X_train_norm = X_train.astype("float32") / 255.0
X_val_norm = X_val.astype("float32") / 255.0
X_test_norm = X_test.astype("float32") / 255.0

y_train = y_train.flatten()
y_val = y_val.flatten()
y_test = y_test.flatten()

print("Data normalized: pixel values are now between 0 and 1.")
print("Random normalized pixel: ", X_train_norm[44998][16][16])

2. First network: Build, train, and test a deep neural network with at least three convolutional layers, two fully connected layers, and two pooling layers

In [ ]:
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# model = Sequential()

# # First Convolution + Pooling
# model.add(Conv2D(32, (5,5), activation='relu', input_shape=(X_train_norm.shape[1], X_train_norm.shape[2], X_train_norm.shape[3])))
# model.add(MaxPooling2D((2,2)))

# # Second Convolution
# model.add(Conv2D(64, (5,5), activation='relu'))


# # Third Convolution + Pooling
# model.add(Conv2D(128, (5,5), activation='relu'))
# model.add(MaxPooling2D((2,2)))

# # Flatten before Dense layers
# model.add(Flatten())

# # Fully connected layers
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))  # Optional: helps prevent overfitting
# model.add(Dense(64, activation='relu'))

# # Output layer (change units based on number of classes)
# model.add(Dense(10, activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Summary of the model
# model.summary()

In [ ]:
# # Compile the model
# model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy, # Define loss function
#                 optimizer= tf.keras.optimizers.Adam(learning_rate=1e-2), metrics=['accuracy']) # Define initial learning rate and metrics.


# # Train the model. Using Colab for training
# history = model.fit(X_train_norm, y_train, # Data feature and data label
#                     batch_size=1024, # Batch size
#                     epochs=10, # Number of training epochs
#                     validation_data=(X_val_norm, y_val)) # Validation set

In [ ]:
# # Visualize training and validation performance
# f,ax=plt.subplots(2,1)

# # Plot training and validation loss
# ax[0].plot(history.history['loss'], color='b',label='Training Loss')
# ax[0].plot(history.history['val_loss'],color='r',label='Validation Loss')

# # Plot training and validation accuracy
# ax[1].plot(history.history['accuracy'],color='b',label='Training Accuracy')
# ax[1].plot(history.history['val_accuracy'],color='r',label='Validation Accuracy')

# plt.legend()

In [ ]:
# # Show the model performance
# result = model.evaluate(X_test_norm, y_test) # If unspecified, batch_size will default to 32
# print(model.metrics_names) # result[0] is loss, result[1] is accuracy. The metrics are defined in dnn_model.complie(...)
# print("Loss and accuracy on the test set: loss = {}, accuracy = {}".format(result[0],result[1]))
# model.summary()

3. Second network: Build, train, and test another deep neural network, with an architecture of your choice, but at most 4M (four million) parameters, ensuring the architecture meets this constraint by verifying with model.summary().

In [ ]:
from keras import backend as K

K = 10  # number of classes

# Build the Sequential model
my_model = Sequential(name="my_model")

# -----------------
# Input layer
# -----------------
my_model.add(Input(shape=(X_train_norm.shape[1], X_train_norm.shape[2], X_train_norm.shape[3])))

# -----------------
# Block 1: Conv + Conv + BN + Pool
# -----------------
my_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
my_model.add(BatchNormalization())
my_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
my_model.add(BatchNormalization())
my_model.add(MaxPooling2D((2, 2)))

# -----------------
# Block 2: Conv + Conv + BN + Pool
# -----------------
my_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
my_model.add(BatchNormalization())
my_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
my_model.add(BatchNormalization())
my_model.add(MaxPooling2D((2, 2)))

# -----------------
# Block 3: Conv + Conv + BN + Pool
# -----------------
my_model.add(Conv2D(256, (5, 5), activation='relu', padding='same'))
my_model.add(BatchNormalization())
my_model.add(Conv2D(256, (5, 5), activation='relu', padding='same'))
my_model.add(BatchNormalization())
my_model.add(MaxPooling2D((2, 2)))


# -----------------
# Flatten + Dropout
# -----------------
my_model.add(Flatten())
my_model.add(Dropout(0.3))

# -----------------
# Dense + Dropout
# -----------------
my_model.add(Dense(512, activation='relu'))
my_model.add(Dropout(0.33))

my_model.add(Dense(256, activation='relu'))
my_model.add(Dropout(0.2))



# -----------------
# Output layer
# -----------------
my_model.add(Dense(K, activation='softmax'))




In [ ]:
# -----------------
# Compile the model
# -----------------
my_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# -----------------
# Model summary
# -----------------
my_model.summary()

In [ ]:
# data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)
datagen.fit(X_train_norm)

In [ ]:
# Train the model. Using Colab for training
model_history = my_model.fit(datagen.flow(X_train_norm, y_train, batch_size=64),
             validation_data=(X_val_norm, y_val),
             epochs=3)

In [ ]:
# Visualize training and validation performance
f,ax=plt.subplots(2,1)

# Plot training and validation loss
ax[0].plot(model_history.history['loss'], color='b',label='Training Loss')
ax[0].plot(model_history.history['val_loss'],color='r',label='Validation Loss')

# Plot training and validation accuracy
ax[1].plot(model_history.history['accuracy'],color='b',label='Training Accuracy')
ax[1].plot(model_history.history['val_accuracy'],color='r',label='Validation Accuracy')

plt.legend()

In [ ]:
# Show the model performance
result = my_model.evaluate(X_test_norm, y_test) # If unspecified, batch_size will default to 32
print(my_model.metrics_names) # result[0] is loss, result[1] is accuracy. The metrics are defined in dnn_model.complie(...)
print("Loss and accuracy on the test set: loss = {}, accuracy = {}".format(result[0],result[1]))

4. Modify the second network architecture by tuning the layer hyperparameters or adjusting the layer design to improve test accuracy while remaining within the four million parameter limit. Discuss your observations and the trade offs of the changes you make.

In [ ]:
# Your code goes here. Please make sure to explain the reasons behind your data processing and modeling choices.
# 1.4

## 2. Open discussion questions

1. How did hyperparameter tuning (learning rate, dropout, batch size) affect your model’s accuracy? Were there any unexpected results?

In [ ]:
# Your argument goes here. Please include data visualization and analysis to back up your argument.
# 2.1

2. How did the constraint of keeping the model within 4 million parameters impact your design choices? Would a larger model necessarily perform better?

In [ ]:
# Your argument goes here. Please include data visualization and analysis to back up your argument.
# 2.2

3. How can deep learning models trained on datasets like CIFAR-10 be applied in real-world scenarios? Give an example.

In [ ]:
# Your argument goes here. Please include data visualization and analysis to back up your argument.
# 2.3

4. Deep learning models for image recognition can have biases. What ethical concerns should be considered when deploying such models?

In [ ]:
# Your argument goes here. Please include data visualization and analysis to back up your argument.
# 2.4

5. What was the most interesting or challenging part of this assignment? If you had more time, what additional improvements would you make?

In [ ]:
# Your argument goes here. Please include data visualization and analysis to back up your argument.
# 2.5